In [1]:
import pandas as pd
import os
import re
from pyhanlp import *
from gensim import corpora, models, similarities
import json

In [2]:
"""创建停用词列表"""
def stopwordslist():
    stopwords = [line.strip() for line in open('./哈工大停用词表.txt',encoding='UTF-8').readlines()]
    return stopwords

In [3]:
def seg_depart(sentence):
    sentence_depart = HanLP.segment(sentence)
    stopwords = stopwordslist()
    outstr = ''
    for word in sentence_depart:
        word = word.toString()
        judge = word.split('/')
        if judge[0] not in stopwords and ('n' in judge[1] or 'v' in judge[1] or 'q' in judge[1]):
            outstr += word
            outstr += " "
    # outstr：'黄蜂 湖人 首发 科比 带伤 战 保罗 加索尔 ...'       
    return outstr

In [5]:
for root, dirs, files in os.walk('../'): 
    for d in dirs:
        if d == '分词统计':
            for file in files:
                if '.xlsx' in file:
                    print(file)
                    outfilename = './' + file.split('.')[0] + '.txt'
                    if not os.path.exists(outfilename):
                        inputs = pd.read_excel('../'+file)
                        outputs = open(outfilename, 'w', encoding='UTF-8')

                        # 把非汉字的字符全部去掉
                        for line in inputs['cpi_content']:
                            line = re.sub(r'[^\u4e00-\u9fa5]+','',str(line))
                            line_seg = seg_depart(line.strip())
                            outputs.write(line_seg.strip() + '\n')

                        outputs.close()
                        print("删除停用词和分词成功！！！")

教育培训.xlsx
旅游交通.xlsx
汽车行业.xlsx
删除停用词和分词成功！！！
网络购物.xlsx
删除停用词和分词成功！！！
电子电器.xlsx
婚恋交友.xlsx
共享服务.xlsx
删除停用词和分词成功！！！
网络服务.xlsx
删除停用词和分词成功！！！
金融保险.xlsx
删除停用词和分词成功！！！
房产家居.xlsx
其它.xlsx
删除停用词和分词成功！！！
物流快递.xlsx


In [6]:
for root, dirs, files in os.walk('../'): 
    for d in dirs:
        if d == '分词统计':
            for file in files:
                print(file)

教育培训.xlsx
旅游交通.xlsx
汽车行业.xlsx
测试.txt
网络购物.xlsx
电子电器.xlsx
.DS_Store
婚恋交友.xlsx
共享服务.xlsx
网络服务.xlsx
金融保险.xlsx
房产家居.xlsx
其它.xlsx
物流快递.xlsx


In [7]:
for root, dirs, files in os.walk('./'): 
    for file in files:
        if '.txt' in file:
            fr = open(file, 'r',encoding='utf-8')
            v_count = {}
            n_count = {}
            for line in fr.readlines():
                lline = line.split(' ')
                for word in lline:
                    if 'v' in word:
                        v_word = word.split('/')[0]
                        try:
                            v_count[v_word] = v_count[v_word] + 1
                        except:
                            v_count[v_word] = 1
                    elif 'n' in word:
                        n_word = word.split('/')[0]
                        try:
                            n_count[n_word] = n_count[n_word] + 1
                        except:
                            n_count[n_word] = 1
            if not os.path.exists(file.split('.')[0]+'名词排名.xlsx'):
                n_count_pd = pd.DataFrame({'名词':list(n_count.keys()), '数量':list(n_count.values())})
                n_count_pd.to_excel(file.split('.')[0]+'名词排名.xlsx', index = False)
            if not os.path.exists(file.split('.')[0]+'动词排名.xlsx'):
                v_count_pd = pd.DataFrame({'动词':list(v_count.keys()), '数量':list(v_count.values())})
                v_count_pd.to_excel(file.split('.')[0]+'动词排名.xlsx', index = False)

In [17]:
n_count_pd = pd.DataFrame({'名词':list(n_count.keys()), '数量':list(n_count.values())})
n_count_pd.sort_values(by='数量',ascending=False)[:10]

,名词,数量
112,月,10352
55,客服,7732
21,合同,7531
113,公司,5762
92,要求,5031
116,押金,4881
6,电话,4658
175,平台,4449
14,问题,4320
11,房子,3881


In [18]:
v_count_pd = pd.DataFrame({'动词':list(v_count.keys()), '数量':list(v_count.values())})
v_count_pd.sort_values(by='数量',ascending=False)[:10]

,动词,数量
38,说,11846
11,没有,10153
113,联系,5116
588,还款,4103
125,退款,3500
162,投诉,3279
106,处理,3164
372,退,2928
91,租房,2863
742,贷,2855


In [1]:
import jpype

#路径
jvmPath = jpype.getDefaultJVMPath() # 获得系统的jvm路径
ext_classpath = r"./ner/hanlp\hanlp-1.6.3.jar:./ner/hanlp"
jvmArg = '-Djava.class.path=' + ext_classpath
jpype.startJVM(jvmPath, jvmArg, "-Xms1g", "-Xmx1g")



/Users/yuchenshen/anaconda3/lib/python3.6/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [2]:
import jpype
jvmPath = jpype.getDefaultJVMPath() # 获得系统的jvm路径


#繁体转简体
def TraditionalChinese2SimplifiedChinese(sentence_str):
    HanLP = jpype.JClass('com.hankcs.hanlp.HanLP')
    return HanLP.convertToSimplifiedChinese(sentence_str)

#切词&命名实体识别与词性标注(可以粗略识别)
def NLP_tokenizer(sentence_str):
    NLPTokenizer = jpype.JClass('com.hankcs.hanlp.tokenizer.NLPTokenizer')
    return NLPTokenizer.segment(sentence_str)

#地名识别，标注为ns
def Place_Recognize(sentence_str):
    HanLP = jpype.JClass('com.hankcs.hanlp.HanLP')
    segment = HanLP.newSegment().enablePlaceRecognize(True)
    return HanLP.segment(sentence_str)

#人名识别,标注为nr
def PersonName_Recognize(sentence_str):
    HanLP = jpype.JClass('com.hankcs.hanlp.HanLP')
    segment = HanLP.newSegment().enableNameRecognize(True)
    return HanLP.segment(sentence_str)

#机构名识别,标注为nt
def Organization_Recognize(sentence_str):
    HanLP = jpype.JClass('com.hankcs.hanlp.HanLP')
    segment = HanLP.newSegment().enableOrganizationRecognize(True)
    return HanLP.segment(sentence_str)

#标注结果转化成列表
def total_result(function_result_input):
    x = str(function_result_input)
    y = x[1:len(x)-1]
    y = y.split(',')
    return y

#时间实体
def time_result(total_result):
    z = []
    for i in range(len(total_result)):
        if total_result[i][-2:] == '/t':
            z.append(total_result[i])
    return z

#Type_Recognition 可以选 ‘place’,‘person’,‘organization’三种实体,
#返回单一实体类别的列表
def single_result(Type_Recognition,total_result):
    if Type_Recognition == 'place':
        Type = '/ns'
    elif Type_Recognition == 'person':
        Type = '/nr'
    elif Type_Recognition == 'organization':
        Type = '/nt'
    else:
        print ('请输入正确的参数：（place，person或organization）')
    z = []
    for i in range(len(total_result)):
        if total_result[i][-3:] == Type:
            z.append(total_result[i])
    return z

#把单一实体结果汇总成一个字典
def dict_result(sentence_str):
    sentence = TraditionalChinese2SimplifiedChinese(sentence_str)
    total_dict = {}
    a = total_result(Place_Recognize(sentence))
    b = single_result('place',a)
    c = total_result(PersonName_Recognize(sentence))
    d = single_result('person',c)
    e = total_result(Organization_Recognize(sentence))
    f = single_result('organization',e)
    g = total_result(NLP_tokenizer(sentence))
    h = time_result(g)
    total_list = [i for i in [b,d,f,h]]
    total_dict.update(place = total_list[0],person = total_list[1],organization = total_list[2],time = total_list[3])
    jpype.shutdownJVM()#关闭JVM虚拟机
    return total_dict

#测试
test_sentence='本人在2017年通过家人缴费9800元到仁和会计学校高新校区学习会计核算班后，在2018年6月30日，该校销售人员又诱导我缴费30000元报名主管会计班、CMA课程，由于我当时毕业不久刚出来工作，没有更多的资金缴费，于是该机构工作人员又诱导并操纵我的手机在智联好期贷APP上贷款交费，让我后期无故承担利息，并导致我征信出现逾期，迫于无奈我只好还完所有款项。 在缴纳该笔3万元用后，该机构没有给我签署任何书面合同，也没有开具任何票据，更也没有给我安排具体的课程，一直到2019年3月打电话询问校区老师，校区老师说还没有开课，后期会通知我，一直到2019年6月还是未收到任何通知及电话，本人根本不知道他们课程如何安排，并且该机构人员流动巨大，报名时的工作人员后续均离职。由于本人有孕在身，身体不佳，没有很多心思来处理，但是时至今日我才发现事情的严重性，我感受到我受到了欺骗。我多次到该机构现场沟通退费事宜，但该机构采用踢皮球方式处理，高新区分校让我找武汉总部，我致电武汉总部又让我找高新分校（我一直联系的是高新区仁和会计培训机构，不知道为何要让我找武汉总部？）。在维权过程中，我发现该机构如下违法事实： 1、与报名人不签订书面合同。在报名后，我惊讶的发现该机构的app上多了一个协议，协议金额只有3200元。我根本不知道这个协议的存在，也没有签过字，根本不是我的意思表示，但协议末尾签署名字居然是机器打印上我的名字。 2、该机构无资质。依据国家法律规定，开设培训机构需要有营业执照和办学资质并且悬挂在营业场所，但是该校区未悬挂相关资质文件。我多次要求提供，其敷衍告知我去找武汉总部要（我报名和联系的一直是成都高新区仁和会计培训机构，为何要求我找总部？）。足以表明该机构是挂靠在仁和会计总部名下，期满社会人士报名学习，却没有办学资质。 3、偷税漏税。该机构诱导我通过智联好期贷APP贷款缴费后，并未开具相关发票和收据。我询问其负责人，其告知“学员没有要求，我们就不会开票”，更是以本月发票不够还在领票中，开票要总部开等各种理由来推迟开发票，直到现在校区还是没有给我开任何发票，足以表明其偷税漏税行为。 4、恶意串通第三方贷款公司。故意诱导学院和第三方平台贷款缴纳学费，未提前告知利息情况。 请：1、仁和会计高新校区退还本人学费。2、请求行政机关依法查处其违法行为。'
print (dict_result(test_sentence))

{'place': [' 高新区/ns', ' 武汉/ns', ' 武汉/ns', ' 高新区/ns', ' 武汉/ns', ' 武汉/ns', ' 成都/ns', ' 高新区/ns'], 'person': [' 仁和/nr', ' 仁和/nr'], 'organization': [], 'time': [' 2017年/t', ' 2018年6月30日/t', ' 当时/t', ' 不久/t', ' 后期/t', ' 2019年3月/t', ' 后期/t', ' 2019年/t', ' 6月/t', ' 本月/t']}
